# Recurrent Neural Networks

## What is Neural Memory?

Neural networks have hidden layers. Normally, the state of your hidden layer is **based ONLY on your input data**.  
So, normally a neural network's information flow would look like this:

![](../images/rnn-description-011.png)

This is straightforward. 

* Certain types of input create certain types of hidden layers. 
* Certain types of hidden layers create certain types of output layers. 

It's kindof a closed system. Memory changes this. Memory means that the hidden layer is a combination of your input data at the current timestep and the hidden layer of the previous timestep.

![](../images/rnn-description-010.png)

Why the hidden layer? Well, we could technically do this. 

![](../images/rnn-description-009.png)

However, we'd be missing out. I encourage you to sit and consider the difference between these two information flows. For a little helpful hint, consider how this plays out. 

Here, we have 4 timesteps of a recurrent neural network pulling information from the previous hidden layer.

![](../images/rnn-description-008.png)

And here, we have 4 timesteps of a recurrent neural network pulling information from the previous input layer

![](../images/rnn-description-007.png)

Maybe, if I colored things a bit, it would become more clear.  
Again, 4 timesteps with **hidden layer recurrence**:

![](../images/rnn-description-005.png)

.... and 4 timesteps with **input layer recurrence....**

![](../images/rnn-description-006.png)


Focus on the last hidden layer (4th line). In the hidden layer recurrence, we see a presence of every input seen so far. In the input layer recurrence, it's exclusively defined by the current and previous inputs. This is why we model hidden recurrence. Hidden recurrence learns what to remember whereas input recurrence is hard wired to just remember the immediately previous datapoint. 

#### What's the practical difference? 
Let's say we were trying to predict the next word in a song given the previous. The "input layer recurrence" would break down if the song accidentally had the same sequence of two words in multiple places. Think about it, if the song had the statements "I love you", and "I love carrots", and the network was trying to predict the next word, how would it know what follows "I love"? It could be carrots. It could be you. The network REALLY needs to know more about what part of the song its in. However, the "hidden layer recurrence" doesn't break down in this way. It subtely remembers everything it saw (with memories becoming more subtle as it they fade into the past). To see this in action, check out [this](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). 

## RNN Types

![Types of Recurrent Neural Networks](../images/diags-rnn.jpeg)

### Each rectangle is a vector and arrows represent functions
* Input vectors are in red
* Output vectors are in blue
* Green vectors hold the RNN's state. 

### From left to right: 
* **Vanilla mode of processing without RNN**, from fixed-sized input to fixed-sized output
 * e.g. image classification
* **Sequence output**
 * e.g. image captioning takes an image and outputs a sentence of words
* **Sequence input**
 * e.g. sentiment analysis where a given sentence is classified as expressing positive or negative sentiment. 
* **Sequence input and sequence output**
 * e.g. Machine Translation: an RNN reads a sentence in English and then outputs a sentence in French
* **Synced sequence input and output**
 * e.g. video classification where we wish to label each frame of the video). 
 
Notice that in every case are no pre-specified constraints on the lengths sequences because the recurrent transformation (green) is fixed and can be applied as many times as we like.

## RNN computation

```python
rnn = RNN()
# x is an input vector, y is the RNN's output vector
y = rnn.step(x) 
```

```python
class RNN:
    # ...
    def step(self, x):
        # update the hidden state
        self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
        # compute the output vector
        y = np.dot(self.W_hy, self.h)
        return y
```

### 2-layer recurrent network

```python
y1 = rnn1.step(x)
y = rnn2.step(y1)
```

We have two separate RNNs:  
One RNN is receiving the input vectors and the second RNN is receiving the output of the first RNN as its input. Except neither of these RNNs know or care - it’s all just vectors coming in and going out, and some gradients flowing through each module during backpropagation.

![The repeating module in a standard RNN contains a single layer.](../images/rnn-description-0012.png)

## Neural Network Memory

![Neural Network Memory](../images/rnn-description-002.gif)

The gif above reflects the magic of recurrent networks, and several very, very important properties.  
It depicts 4 timesteps. 

* The first is exclusively influenced by the input data. 
* The second one is a mixture of the first and second inputs. 
* This continues on. 

You should recognize that, in some way, network 4 is "full".  
Presumably, timestep 5 would have to choose which memories to keep and which ones to overwrite.
This is very real. It's the notion of memory "capacity".  
As you might expect, bigger layers can hold more memories for a longer period of time.  
Also, this is when the network learns to **forget irrelevant memories** and **remember important memories**. 

* What significant thing do you notice in timestep 3? 
* Why is there more green in the hidden layer than the other colors?

Also notice that the hidden layer is the barrier between the input and the output.  
**In reality, the output is no longer a pure function of the input**.  
The input is just changing what's in the memory, and the output is exclusively based on the memory. Another interesting takeaway.  
If there was no input at timesteps 2, 3, and 4, the hidden layer would still change from timestep to timestep.

## Backpropagation Through Time

![Backpropagation Through Time with RNN](../images/rnn-description-003.gif)

* Black is the prediction
* Errors are bright yellow
* Derivatives are mustard colored.

They learn by fully propagating forward from 1 to 4 (through an entire sequence of arbitrary length), and then backpropagating all the derivatives from 4 back to 1. You can also pretend that it's just a funny shaped normal neural network, except that we're re-using the same weights (synapses 0,1,and h) in their respective places. Other than that, it's normal backpropagation. 

## Our Toy Code

We're going to be using a recurrent neural network to model binary addition.  
Do you see the sequence below? **What do the colored ones in squares at the top signify?**

![recurrent neural network to model binary addition](../images/rnn-description-004.gif)

The colorful 1s in boxes at the top signify the "carry bit". They "carry the one" when the sum overfows at each place. This is the tiny bit of memory that we're going to teach our neural network how to model. It's going to "carry the one" when the sum requires it. (click [here](https://www.youtube.com/watch?v=jB_sRh5yoZk) to learn about when this happens)

So, binary addition moves from right to left, where we try to predict the number beneath the line given the numbers above the line. We want the neural network to move along the binary sequences and remember when it has carried the 1 and when it hasn't, so that it can make the correct prediction. 

Don't get too caught up in the problem. The network actually doesn't care too much. Just recognize that we're going to have two inputs at each time step, (either a one or a zero from each number begin added). These two inputs will be propagated to the hidden layer, which will have to remember whether or not we carry. The prediction will take all of this information into account to predict the correct bit at the given position (time step).

Source: [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) &[Anyone Can Learn To Code an LSTM-RNN in Python](https://iamtrask.github.io/2015/11/15/anyone-can-code-lstm/)